In [49]:
import numpy as np
import pandas as pd
import scipy 
import scipy.io as sio
from scipy.optimize import linprog

# 0.Data Preparation

In [50]:
# set the data
dataFile = "Vehicle_data.mat"
data = sio.loadmat(dataFile)
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Apr 27 23:03:20 2018',
 '__version__': '1.0',
 '__globals__': [],
 'D': array([[3.8, 3. , 3.3, 5. , 5.5, 3.5, 4.1, 5.3, 5.3, 6.4, 3.8, 3.3, 3.5,
         5.3],
        [3.8, 3. , 3.3, 5. , 5.5, 3.5, 4.1, 5.3, 5.3, 6.4, 3.8, 3.3, 3.5,
         5.3],
        [3.8, 3. , 3.3, 5. , 5.5, 3.5, 4.1, 5.3, 5.3, 6.4, 3.8, 3.3, 3.5,
         5.3]]),
 'E': array([[47.5,  0. ,  0. ],
        [ 0. , 20.3,  0. ],
        [ 0. ,  0. , 45. ]]),
 'F': array([[47.12854486, 31.14713143, 24.77254829, 28.85423571, 28.37839857,
         28.81796386, 47.10785029, 26.723036  , 27.373036  , 27.67232514,
         28.8654937 , 24.1395787 , 28.4449025 , 26.2191487 ],
        [20.84712843, 13.50518857, 11.51685293, 13.39599643, 13.48221821,
         13.20180725, 19.61666886, 13.03205814, 13.68205814, 13.96160229,
         13.07285545, 10.95329983, 12.89818063, 12.59776983],
        [35.99748757, 23.67209143, 19.15686907, 22.31120357, 22.0748917

In [51]:
data['Q']=np.dot(data['Qb'],np.ones((3,14)))
data['Q']

array([[42.75, 42.75, 42.75, 42.75, 42.75, 42.75, 42.75, 42.75, 42.75,
        42.75, 42.75, 42.75, 42.75, 42.75],
       [16.24, 16.24, 16.24, 16.24, 16.24, 16.24, 16.24, 16.24, 16.24,
        16.24, 16.24, 16.24, 16.24, 16.24],
       [31.5 , 31.5 , 31.5 , 31.5 , 31.5 , 31.5 , 31.5 , 31.5 , 31.5 ,
        31.5 , 31.5 , 31.5 , 31.5 , 31.5 ]])

# 1.Organize data

In [52]:
data['Qd']=np.dot(data['r'],data['D']) 
# electricity consumed on the road of Car i

data['Qb']=np.dot((np.eye(3)-data['q']),data['E'])
# electricity need of Car i

data['Q']=np.dot(data['Qb'],np.ones((3,14)))+data['Qd'] 
# Total electricity need of Car i

data['Tc']=np.array([[12.40537143, 12.36514286, 12.38022857, 12.46571429, 12.49085714,
         12.39028571, 12.42045714, 12.4808    , 12.4808    , 12.53611429,
          1.08547   ,  1.08327   ,  1.08415   ,  1.09207   ],
        [ 4.79308571,  4.76085714,  4.77294286,  4.84142857,  4.86157143,
          4.781     ,  4.80517143,  4.85351429,  4.85351429,  4.89782857,
          0.419395  ,  0.4176325 ,  0.4183375 ,  0.4246825 ],
        [ 9.18131429,  9.14314286,  9.15745714,  9.23857143,  9.26242857,
          9.167     ,  9.19562857,  9.25288571,  9.25288571,  9.30537143,
          0.803365  ,  0.8012775 ,  0.8021125 ,  0.8096275 ]])
#                  data['Q'] / data['p']
# time needed for charging of Car i in Station j 

data['Pc']=np.dot(np.ones((3,1)),data['pc'])*data['Q']
# Charging fee of Car i in Station j


data['Ps']=np.dot(np.ones((3,1)),data['ps'])*data['Q']
# Service fee of Car i in Station j


data['Pp']=np.dot(np.ones((3,1)),data['pp'])*data['Tc']
# Parking fee of Car i in Station j


data['P']=data['Pc']+data['Ps']+data['Pp'] 
# Total fee of Car i in Station j

# 2.Set the goal function

According to the utility, give the weights

In [53]:
# Normal Charging
F1=0.3*data['D']+0.13*data['Td']+0.01*data['Tc']+0.3*data['P']
-0.13*np.dot(np.ones((3,1)),data['S'])-0.13*np.ones((3,1))*data['R']

# Urgent Charging
F2=0.3*data['D']+0.1*data['Td']+0.3*data['Tc']+0.1*data['P']
-0.1*np.dot(np.ones((3,1)),data['S'])-0.11*np.ones((3,1))*data['R']

# Rush Hour
F3=0.1*data['D']+0.3*data['Td']+0.3*data['Tc']+0.1*data['P']
-0.3*np.dot(np.ones((3,1)),data['S'])-0.11*np.ones((3,1))*data['R']

# Shortest Distance
F4=data['D'] 

# Arrive earliest
F5=data['Td'] 

# Finish Charge earliest
F6=data['Tc'] 

# Cheapest
F7=data['P'] 

# Hightest Score(Highest Satisfaction)
F8=np.dot(-np.ones((3,1)),data['S'] )

# Most spot
F9=np.dot(-np.ones((3,1)),data['R'] )

# 3.Set the parameters of Linear Programming

In [54]:
# build a function which make it easier to iterate every goal function 
# function could generate every parameters


def parameters(goal):

    C=goal

    # calculate the row
    n=C.shape[0]

    # calculate the column
    m=C.shape[1]

    # Calculate the goal function
    c=C.flatten('F')

    # set the unequation restriction
    Qz=np.dot(data['r'],data['D'])-np.dot(np.dot(data['q'],data['E']),np.ones((n,m)))
    Qz=Qz.flatten('F')
    a=np.zeros((m,n*m))

    for i in range(0,m):
        for j in range(i*n, n*(i+1)):
                       a[i:i+1,j:j+1]=1        

    Qz=Qz*np.ones((1,n*m))
    A=np.vstack((a,Qz))

    # unequation coefficient 
    B=np.vstack((data['R'].conj().T,np.zeros((1,1))))


    # set the equation restriction
    Ae = np.zeros((n,n*m))
    for i in range(0,n):
         for k in range(i, n*m, n):
                Ae[i:i+1,k:k+1]=1 

    # equation coefficient             
    Be=np.ones((n,1))
    
    return (c,A,B,Ae,Be)

# 4.Modelling and get answer

In [55]:
func = [F1,F2,F3,F4,F5,F6,F7,F8,F9]
funcname = ['F1','F2','F3','F4','F5','F6','F7','F8','F9']

In [56]:
i=0
for goal in func:
    (c,A,B,Ae,Be) = parameters(goal)
    res=linprog(c,A,B,Ae,Be,bounds = (0,1))
    print("---")
    print(funcname[i])
    print("---")
    print(res)
    print("***********************")
    print("")
    print("")
    i+=1

---
F1
---
     fun: 51.82330069999999
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([11.    ,  5.    ,  3.    ,  3.    ,  5.    ,  8.    ,  1.    ,
        5.    ,  9.    ,  6.    ,  2.    ,  0.    ,  6.    ,  1.    ,
       20.7128,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,
        1.    ,  0.    ,  0.    ,  1.    ,  1.    ,  1.    ,  1.    ,
        1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,
        1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,
        1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  0.    ,
        1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,
        1.    ])
  status: 0
 success: True
       x: array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0.])
***********************


---
F2
---
     fun: 23.225799000000002
 message: 'Optimiza